In [42]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *

yu.setpath('analysis_avgx')

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
enss=['b','c','d']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
path=f'pkl/analysis_c2pt/reg_ignore/ens2pars_jk_meffnst_selected.pkl'
[ens2pars_jk_meff1st,ens2pars_jk_meff2st,ens2pars_jk_meff3st]=yu.load_pkl(path)

# conn

In [31]:
key2tf2ratio={}
for ens in enss:
    key2tf2ratio[(ens,'j+;conn')]={}
    key2tf2ratio[(ens,'j-;conn')]={}
    
    basepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{yu.ens2full[ens]}/data_merge/'
    
    mN_jk=ens2pars_jk_meff2st[ens][:,0]
    factor_equal=1/(-3*mN_jk/4)
    
    path=f'{basepath}/conn_2pt.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0])
        
        tf2c2pt={}
        for tf in f['data'].keys():
            t=f[f'data/{tf}'][:]
            t=yu.jackknife(np.real(t[:,:,imom]))
            tf2c2pt[int(tf)]=t

    path=f'{basepath}/conn_0,0,0,0,0,0.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0,0,0,0])
        
        for jtf in f['data'].keys():
            j,tf=jtf.split('_'); tf=int(tf)
            t=f[f'data/{jtf}'][:]
            t=t[:,:,0,projs.index('P0'),inserts.index('tt')]
            c3pt=yu.jackknife(t)
            ratio=np.real(c3pt/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
            key=(ens,j)
            key2tf2ratio[key][tf]=ratio
            
ens2tfs_conn={}
for ens in enss:
    tfs=list(key2tf2ratio[(ens,'j+;conn')].keys()); tfs.sort()
    ens2tfs_conn[ens]=tfs
    print(ens,tfs)

b [8, 10, 12, 14, 16, 18, 20]
c [6, 8, 10, 12, 14, 16, 18, 20, 22]
d [8, 10, 12, 14, 16, 18, 20, 22, 24, 26]


In [40]:
keys=[(ens,j) for j in ['j+;conn','j-;conn'] for ens in enss]
xunits=[yu.ens2a[ens] for j in ['j+;conn','j-;conn'] for ens in enss]
key2fits={}

for i,key in enumerate(keys):
    print(f'{i}/{len(keys)}',key,end='               \r')
    ens,j=key
    tf2ratio=key2tf2ratio[key]
    
    pars_jk_meff2st=ens2pars_jk_meff2st[ens]
    tfmins=range(8,ens2tfs_conn[ens][-1],2); tcmins=range(2,ens2tfs_conn[ens][-1]//2-1)
    fits=yu.doFit_3ptSym_2st2step(tf2ratio,tfmins,tcmins,pars_jk_meff2st,downSampling=[1,1],symmetrizeQ=False,label=f'{ens}_{j}')
    key2fits[key]=fits
    
list_tf2ratio_fits=[[key2tf2ratio[key],key2fits[key],ens2pars_jk_meff2st[key[0]]] for key in keys]
fig,axs,result=yu.makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunits=xunits,tcmin_rainbow=2)
for i,key in enumerate(keys):
    axs[i,0].set_ylabel(key)
yu.finalizePlot('conn')

key2bare={}
for i,key in enumerate(keys):
    res=result[i][:,0]
    key2bare[key]=res
yu.save_pkl_reg('key2bare_conn',key2bare)